# NEWS2PROFIT - Data Exploration and Analysis

This notebook demonstrates how to use the NEWS2PROFIT project for stock market prediction using machine learning and financial news sentiment analysis.

## Project Overview
- **Objective**: Predict NSE stock movement (UP/DOWN/NEUTRAL) using historical data and news sentiment
- **Data Sources**: yfinance for stock data, NewsAPI/Twitter/Reddit for news
- **ML Models**: Logistic Regression, XGBoost, LSTM
- **Features**: Technical indicators (SMA, EMA, RSI, MACD) + sentiment scores

## Notebook Structure
1. Import libraries and setup
2. Data collection from multiple sources
3. Data preprocessing and feature engineering
4. Sentiment analysis of news data
5. Model training and evaluation
6. Visualization and insights

In [7]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Import NEWS2PROFIT modules with error handling
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

try:
    from config.config import *
    print(" Config imported successfully!")
except ImportError as e:
    print(f"Config import error: {e}")

try:
    from src.data_collection import StockDataCollector, NewsDataCollector
    print(" Data collection modules imported successfully!")
except ImportError as e:
    print(f" Data collection import error: {e}")

try:
    from src.preprocessing import StockDataPreprocessor, NewsDataPreprocessor, DataIntegrator
    print("Preprocessing modules imported successfully!")
except ImportError as e:
    print(f"Preprocessing import error: {e}")

try:
    from src.sentiment import SentimentAnalyzer
    print("Sentiment analysis module imported successfully!")
except ImportError as e:
    print(f" Sentiment analysis import error: {e}")
    print("Try installing missing dependencies: pip install textblob vaderSentiment")

try:
    from src.model import StockPredictor, LogisticRegressionModel
    print("Model modules imported successfully!")
except ImportError as e:
    print(f" Model import error: {e}")

# Set plotting style
try:
    plt.style.use('seaborn-v0_8')
    sns.set_palette("husl")
    print(" Plotting style set successfully!")
except Exception as e:
    print(f" Plotting style warning: {e}")
    # Fallback to default style
    sns.set_palette("deep")

print(" All available libraries imported!")
try:
    print(f" Selected stocks: {NSE_STOCKS[:5]}...")  # Show first 5 stocks
except NameError:
    print(" Config not loaded - will use default stock selection")

 Config imported successfully!
 Data collection modules imported successfully!
Preprocessing modules imported successfully!
 Sentiment analysis import error: No module named 'textblob'
Try installing missing dependencies: pip install textblob vaderSentiment
 Model import error: No module named 'xgboost'
 Plotting style set successfully!
 All available libraries imported!
 Selected stocks: ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS']...


## 1. Data Collection

First, let's collect stock data from yfinance and news data from various sources.

In [8]:
# Collect Stock Data
print("Collecting stock data...")

# Check if required modules are available
try:
    # Initialize stock data collector
    stock_collector = StockDataCollector()
    print("StockDataCollector initialized successfully!")
    
    # Focus on 5 specific NSE stocks for analysis
    selected_stocks = ['RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'HDFCBANK.NS', 'ICICIBANK.NS']
    
    # Fetch 6 months of historical data
    stock_data_dict = stock_collector.fetch_stock_data(selected_stocks, period="6mo")
    
    # Combine all stock data into a single DataFrame
    all_stock_data = []
    for symbol, data in stock_data_dict.items():
        if data is not None and not data.empty:
            all_stock_data.append(data)
    
    if all_stock_data:
        combined_stock_data = pd.concat(all_stock_data, ignore_index=True)
        print(f"Collected stock data: {combined_stock_data.shape}")
        print(f"Date range: {combined_stock_data['date'].min()} to {combined_stock_data['date'].max()}")
        print(f"Stocks: {combined_stock_data['symbol'].unique()}")
        
        # Save stock data to CSV
        stock_collector.save_stock_data()
        print("Stock data saved to CSV files")
    else:
        print("No stock data collected")
        combined_stock_data = pd.DataFrame()
        
except NameError as e:
    print(f"Import error: {e}")
    print("Please run cell 2 first to import all required modules")
    # Create empty DataFrame as fallback
    import pandas as pd
    combined_stock_data = pd.DataFrame()
    print("Created empty DataFrame as fallback")
    
except Exception as e:
    print(f"Error collecting stock data: {e}")
    # Create empty DataFrame as fallback
    import pandas as pd
    combined_stock_data = pd.DataFrame()
    print("Created empty DataFrame as fallback")

INFO:src.data_collection:Fetching stock data for 5 symbols


StockDataCollector initialized successfully!


INFO:src.data_collection:Successfully fetched 127 records for RELIANCE.NS
INFO:src.data_collection:Successfully fetched 127 records for TCS.NS
INFO:src.data_collection:Successfully fetched 127 records for TCS.NS
INFO:src.data_collection:Successfully fetched 127 records for INFY.NS
INFO:src.data_collection:Successfully fetched 127 records for INFY.NS
INFO:src.data_collection:Successfully fetched 127 records for HDFCBANK.NS
INFO:src.data_collection:Successfully fetched 127 records for HDFCBANK.NS
INFO:src.data_collection:Successfully fetched 127 records for ICICIBANK.NS
INFO:src.data_collection:Saved RELIANCE.NS data to data\raw\RELIANCE_NS_historical.csv
INFO:src.data_collection:Successfully fetched 127 records for ICICIBANK.NS
INFO:src.data_collection:Saved RELIANCE.NS data to data\raw\RELIANCE_NS_historical.csv
INFO:src.data_collection:Saved TCS.NS data to data\raw\TCS_NS_historical.csv
INFO:src.data_collection:Saved INFY.NS data to data\raw\INFY_NS_historical.csv
INFO:src.data_collec

Collected stock data: (635, 9)
Date range: 2025-04-11 to 2025-10-10
Stocks: ['RELIANCE.NS' 'TCS.NS' 'INFY.NS' 'HDFCBANK.NS' 'ICICIBANK.NS']
Stock data saved to CSV files


In [9]:
# Collect News Data for 5 Specific Stocks
print("Collecting news data for RELIANCE, TCS, INFY, HDFCBANK, ICICIBANK...")

# Check if required modules are available
try:
    # Initialize news data collector
    news_collector = NewsDataCollector()
    print("NewsDataCollector initialized successfully!")
    
    # Fetch news data from the last 30 days for specific stocks + business headlines
    # This will automatically save data to separate CSV files for each source
    news_data = news_collector.fetch_all_news_data(days_back=30, save_to_csv=True)
    
    if not news_data.empty:
        print(f"Collected news data: {news_data.shape}")
        print(f"News sources: {news_data['source'].value_counts().to_dict()}")
        print(f"Date range: {news_data['date'].min()} to {news_data['date'].max()}")
        print(f"Keywords covered: {news_data['keyword'].unique()[:10]}")  # Show first 10 keywords
        print("News data saved to separate CSV files by source (NewsAPI, Twitter, Reddit)")
    else:
        print("No news data collected. Check API keys in .env file.")
        
except NameError as e:
    print(f"Import error: {e}")
    print("Please run cell 2 first to import all required modules")
    # Create empty DataFrame as fallback
    import pandas as pd
    news_data = pd.DataFrame()
    print("Created empty DataFrame as fallback")
    
except Exception as e:
    print(f"News collection failed: {str(e)}")
    print("Tip: Make sure API keys are properly set in .env file")
    # Create empty DataFrame for demo purposes
    import pandas as pd
    news_data = pd.DataFrame()
    
# Display what files were created
import os
if os.path.exists('data/raw'):
    print("\nCSV files created:")
    for file in os.listdir('data/raw'):
        if file.endswith('.csv'):
            print(f" {file}")
else:
    print("\n No data directory found - will be created when data is collected")

INFO:src.data_collection:Twitter API v2 initialized successfully with Bearer Token
INFO:src.data_collection:Reddit API initialized successfully
INFO:src.data_collection:Fetching news from all sources for 5 specific stocks...
INFO:src.data_collection:Fetching NewsAPI data...
INFO:src.data_collection:Fetching stock news from SerpApi Google News Light...
INFO:src.data_collection:Fetching news for RELIANCE...
INFO:src.data_collection:Reddit API initialized successfully
INFO:src.data_collection:Fetching news from all sources for 5 specific stocks...
INFO:src.data_collection:Fetching NewsAPI data...
INFO:src.data_collection:Fetching stock news from SerpApi Google News Light...
INFO:src.data_collection:Fetching news for RELIANCE...


NewsDataCollector initialized successfully!


INFO:src.data_collection:Fetched 20 articles for RELIANCE
INFO:src.data_collection:Fetching news for TCS...
INFO:src.data_collection:Fetching news for TCS...
INFO:src.data_collection:Fetched 20 articles for TCS
INFO:src.data_collection:Fetched 20 articles for TCS
INFO:src.data_collection:Fetching news for INFY...
INFO:src.data_collection:Fetching news for INFY...
INFO:src.data_collection:Fetched 20 articles for INFY
INFO:src.data_collection:Fetched 20 articles for INFY
INFO:src.data_collection:Fetching news for HDFCBANK...
INFO:src.data_collection:Fetching news for HDFCBANK...
INFO:src.data_collection:Fetched 20 articles for HDFCBANK
INFO:src.data_collection:Fetched 20 articles for HDFCBANK
INFO:src.data_collection:Fetching news for ICICIBANK...
INFO:src.data_collection:Fetching news for ICICIBANK...
INFO:src.data_collection:Fetched 20 articles for ICICIBANK
INFO:src.data_collection:Fetched 20 articles for ICICIBANK
INFO:src.data_collection:Total articles fetched: 100
INFO:src.data_col

Collected news data: (438, 10)
News sources: {'reddit': 342, 'serpapi_google_news': 96}
Date range: 2025-09-10 to 2025-10-10
Keywords covered: ['RELIANCE' 'TCS' 'INFY' 'HDFCBANK' 'ICICIBANK' 'IndiaInvestments'
 'indianstreetbets' 'StockMarket' 'investing']
News data saved to separate CSV files by source (NewsAPI, Twitter, Reddit)

CSV files created:
 combined_data_20251009_211008.csv
 combined_data_20251009_211630.csv
 combined_data_20251010_092303.csv
 combined_data_20251010_095044.csv
 HDFCBANK_NS_historical.csv
 ICICIBANK_NS_historical.csv
 INFY_NS_historical.csv
 newsapi_data_20251009_210956.csv
 newsapi_data_20251009_211605.csv
 newsapi_data_20251010_092250.csv
 newsapi_data_20251010_095032.csv
 reddit_data_20251009_211008.csv
 reddit_data_20251009_211630.csv
 reddit_data_20251010_092302.csv
 reddit_data_20251010_095044.csv
 RELIANCE_NS_historical.csv
 TCS_NS_historical.csv


## 2. Data Exploration and Visualization

Let's explore the collected stock data with some visualizations.

In [10]:
# Display basic information about stock data
# Check if combined_stock_data exists and handle gracefully
try:
    if 'combined_stock_data' in locals() and not combined_stock_data.empty:
        print("Stock Data Overview")
        print("=" * 40)
        print(f"Shape: {combined_stock_data.shape}")
        print(f"Columns: {list(combined_stock_data.columns)}")
        print(f"Memory usage: {combined_stock_data.memory_usage().sum() / 1024:.2f} KB")
        
        # Display first few rows
        print("\n Sample Data:")
        display(combined_stock_data.head())
        
        # Basic statistics
        print("\n Basic Statistics:")
        display(combined_stock_data.describe())
    else:
        print(" No stock data available to display")
        print("\n To fix this issue:")
        print("1. Run Cell 4 (Data Collection) first to collect stock data")
        print("2. Make sure the stock data collection completed successfully")
        print("3. Then run this cell again")
        
        # Check if we can load existing data from files
        import os
        import pandas as pd
        
        raw_data_path = "data/raw"
        if os.path.exists(raw_data_path):
            csv_files = [f for f in os.listdir(raw_data_path) if f.endswith('_historical.csv')]
            if csv_files:
                print(f"\n Found {len(csv_files)} existing stock data files:")
                for file in csv_files:
                    print(f"   - {file}")
                print("\nYou can load existing data by running the data collection cell.")
            else:
                print("\n No existing stock data files found in data/raw/")
        else:
            print(f"\n Data directory '{raw_data_path}' not found")

except NameError:
    print(" combined_stock_data is not defined")
    print("\n To fix this:")
    print("1. First run Cell 2 (Import Required Libraries)")
    print("2. Then run Cell 4 (Collect Stock Data)")
    print("3. Finally run this cell again")
    print("\n Make sure to run cells in order for the best experience!")

except Exception as e:
    print(f"Unexpected error: {e}")
    print("Please check if all previous cells ran successfully.")

Stock Data Overview
Shape: (635, 9)
Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'dividends', 'stock_splits', 'symbol']
Memory usage: 44.78 KB

 Sample Data:


,date,open,high,low,close,volume,dividends,stock_splits,symbol
0,2025-04-11,1190.395702,1217.786306,1190.395702,1214.100952,11224242,0.0,0.0,RELIANCE.NS
1,2025-04-15,1246.023534,1246.023534,1232.178804,1235.166870,13378535,0.0,0.0,RELIANCE.NS
2,2025-04-16,1229.190730,1235.963728,1224.111042,1234.370117,8248464,0.0,0.0,RELIANCE.NS
3,2025-04-17,1235.266450,1275.107331,1222.716597,1269.430054,15398401,0.0,0.0,RELIANCE.NS
4,2025-04-21,1264.947876,1295.724978,1261.959810,1290.346436,10429964,0.0,0.0,RELIANCE.NS



 Basic Statistics:


,open,high,low,close,volume,dividends,stock_splits
count,635.000000,635.000000,635.000000,635.000000,6.350000e+02,635.000000,635.000000
mean,1711.049285,1723.857827,1697.908866,1710.720933,9.901257e+06,0.146457,0.003150
std,795.128223,800.481667,787.856702,793.643563,7.235555e+06,1.670811,0.079368
min,875.540352,897.567623,875.540352,891.326965,0.000000e+00,0.000000,0.000000
25%,1371.400024,1381.099976,1362.000000,1372.200012,4.819321e+06,0.000000,0.000000
50%,1427.882387,1437.805844,1418.733726,1428.199951,8.473741e+06,0.000000,0.000000
75%,1577.150024,1591.012686,1559.678453,1573.700012,1.321288e+07,0.000000,0.000000
max,3576.367006,3586.344199,3502.970758,3576.366943,5.008483e+07,30.000000,2.000000


In [ ]:
# Stock Price and Volume Visualization
# Check if combined_stock_data exists and handle gracefully
try:
    if 'combined_stock_data' not in locals() or combined_stock_data.empty:
        print("combined_stock_data is not available for visualization")
        print("\n To fix this:")
        print("1. Run Cell 2 (Import Required Libraries) first")
        print("2. Run Cell 4 (Collect Stock Data) to collect stock data")
        print("3. Make sure the data collection completed successfully")
        print("4. Then run this cell again")
        
        # Try to load existing data files as fallback
        import os
        import pandas as pd
        
        print("\n Checking for existing data files...")
        raw_data_path = "data/raw"
        
        if os.path.exists(raw_data_path):
            csv_files = [f for f in os.listdir(raw_data_path) if f.endswith('_historical.csv')]
            if csv_files:
                print(f" Found {len(csv_files)} existing stock data files:")
                for file in csv_files:
                    print(f"   - {file}")
                
                # Try to load and combine existing data
                try:
                    print("\n Attempting to load existing data...")
                    all_data = []
                    for file in csv_files:
                        file_path = os.path.join(raw_data_path, file)
                        df = pd.read_csv(file_path)
                        if not df.empty:
                            all_data.append(df)
                    
                    if all_data:
                        combined_stock_data = pd.concat(all_data, ignore_index=True)
                        print(f" Successfully loaded {combined_stock_data.shape[0]} rows of stock data")
                        print(f"Stocks: {combined_stock_data['symbol'].unique()}")
                        print("Now proceeding with visualization...")
                    else:
                        print(" No valid data found in existing files")
                        combined_stock_data = pd.DataFrame()
                        
                except Exception as e:
                    print(f" Error loading existing data: {e}")
                    combined_stock_data = pd.DataFrame()
            else:
                print(" No existing stock data files found")
                combined_stock_data = pd.DataFrame()
        else:
            print(f" Data directory '{raw_data_path}' not found")
            combined_stock_data = pd.DataFrame()

    # Proceed with visualization if data is available
    if 'combined_stock_data' in locals() and not combined_stock_data.empty:
        print(f"\n Creating visualization for {len(combined_stock_data['symbol'].unique())} stocks...")
        
        # Interactive price chart using Plotly
        fig = make_subplots(
            rows=2, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.1,
            subplot_titles=('Stock Prices', 'Trading Volume'),
            row_width=[0.3, 0.7]
        )

        # Add price traces for each stock
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

        for i, symbol in enumerate(combined_stock_data['symbol'].unique()):
            stock_subset = combined_stock_data[combined_stock_data['symbol'] == symbol]
            color = colors[i % len(colors)]

            # Price line
            fig.add_trace(
                go.Scatter(
                    x=stock_subset['date'], 
                    y=stock_subset['close'],
                    mode='lines',
                    name=symbol.replace('.NS', ''),
                    line=dict(color=color, width=2)
                ),
                row=1, col=1
            )

            # Volume bars
            fig.add_trace(
                go.Bar(
                    x=stock_subset['date'],
                    y=stock_subset['volume'],
                    name=f"{symbol.replace('.NS', '')} Volume",
                    marker_color=color,
                    opacity=0.7,
                    showlegend=False
                ),
                row=2, col=1
            )

        fig.update_layout(
            title='NSE Stock Prices and Volume',
            height=600,
            hovermode='x unified'
        )

        fig.update_yaxes(title_text="Price (₹)", row=1, col=1)
        fig.update_yaxes(title_text="Volume", row=2, col=1)
        fig.update_xaxes(title_text="Date", row=2, col=1)

        fig.show()
        
        print(" Visualization completed successfully!")
    else:
        print("\n No stock data available for visualization")
        print(" Please run the data collection cell (Cell 4) first")

except Exception as e:
    print(f" Unexpected error: {e}")
    print(" Please ensure all previous cells ran successfully")
    import traceback
    traceback.print_exc()


📈 Creating visualization for 5 stocks...


✅ Visualization completed successfully!


## 3. Data Preprocessing and Feature Engineering

Now let's preprocess the stock data and create technical indicators for machine learning.

In [12]:
# Preprocess stock data and create technical indicators
try:
    if 'combined_stock_data' in locals() and not combined_stock_data.empty:
        print("Processing stock data...")
        
        # Initialize stock preprocessor
        stock_preprocessor = StockDataPreprocessor()
        
        # Step 1: Clean the data
        cleaned_stock_data = stock_preprocessor.clean_stock_data(combined_stock_data)
        print(f"Cleaned data shape: {cleaned_stock_data.shape}")
        
        # Step 2: Create technical indicators
        stock_with_indicators = stock_preprocessor.create_technical_indicators(cleaned_stock_data)
        print(f"Added technical indicators. New shape: {stock_with_indicators.shape}")
        
        # Step 3: Create target variable for prediction
        stock_with_target = stock_preprocessor.create_target_variable(stock_with_indicators)
        print(f"Created target variable. Final shape: {stock_with_target.shape}")
        
        # Show target distribution
        print("\n Target Distribution (Next Day Movement):")
        target_counts = stock_with_target['target'].value_counts()
        print(target_counts)
        
        # Show some technical indicators
        tech_indicators = [col for col in stock_with_indicators.columns if col in [
            'sma_5', 'sma_20', 'ema_12', 'ema_26', 'rsi', 'macd', 'daily_return', 'volatility'
        ]]
        
        print(f"\n Technical indicators created: {tech_indicators}")
        
        # Display sample of processed data
        print("\n Sample of processed data with indicators:")
        display(stock_with_indicators[['date', 'symbol', 'close', 'daily_return', 'sma_5', 'sma_20', 'rsi']].head(10))
        
    else:
        print(" No stock data to process")
        print(" Please run the data collection cell first")
        import pandas as pd
        stock_with_target = pd.DataFrame()
        
except NameError as e:
    print(f"Import error: {e}")
    print("Please run cell 2 first to import all required modules")
    import pandas as pd
    stock_with_target = pd.DataFrame()
    
except Exception as e:
    print(f" Error processing stock data: {e}")
    import pandas as pd
    stock_with_target = pd.DataFrame()

INFO:src.preprocessing:Cleaning stock data...
INFO:src.preprocessing:Cleaned data shape: (635, 9)
INFO:src.preprocessing:Cleaned data shape: (635, 9)
INFO:src.preprocessing:Creating technical indicators...
INFO:src.preprocessing:Creating technical indicators...
INFO:src.preprocessing:Created technical indicators. New shape: (635, 56)
INFO:src.preprocessing:Creating target variable...
INFO:src.preprocessing:Target distribution:
target
DOWN       313
UP         307
NEUTRAL     15
Name: count, dtype: int64
INFO:src.preprocessing:Converting to binary classification (UP/DOWN) due to insufficient NEUTRAL samples
INFO:src.preprocessing:Target distribution:
target
DOWN       313
UP         307
NEUTRAL     14
Name: count, dtype: int64
INFO:src.preprocessing:Created technical indicators. New shape: (635, 56)
INFO:src.preprocessing:Creating target variable...
INFO:src.preprocessing:Target distribution:
target
DOWN       313
UP         307
NEUTRAL     15
Name: count, dtype: int64
INFO:src.preproce

Processing stock data...
Cleaned data shape: (635, 9)
Added technical indicators. New shape: (635, 56)
Created target variable. Final shape: (634, 64)

 Target Distribution (Next Day Movement):
target
DOWN       313
UP         307
NEUTRAL     14
Name: count, dtype: int64

 Technical indicators created: ['daily_return', 'sma_5', 'sma_20', 'ema_12', 'ema_26', 'macd', 'rsi', 'volatility']

 Sample of processed data with indicators:


,date,symbol,close,daily_return,sma_5,sma_20,rsi
0,2025-04-11,RELIANCE.NS,1214.100952,NaN,NaN,NaN,NaN
381,2025-04-11,HDFCBANK.NS,891.326965,-0.265854,NaN,NaN,NaN
254,2025-04-11,INFY.NS,1389.894043,0.559354,NaN,NaN,NaN
127,2025-04-11,TCS.NS,3192.197021,1.296720,NaN,NaN,NaN
508,2025-04-11,ICICIBANK.NS,1301.606689,-0.592254,1597.825134,NaN,NaN
382,2025-04-15,HDFCBANK.NS,920.014221,-0.293170,1539.007788,NaN,NaN
128,2025-04-15,TCS.NS,3208.693848,2.487657,2002.481165,NaN,NaN
1,2025-04-15,RELIANCE.NS,1235.166870,-0.615056,1971.535730,NaN,NaN
509,2025-04-15,ICICIBANK.NS,1339.861450,0.084761,1601.068616,NaN,NaN
255,2025-04-15,INFY.NS,1406.214355,0.049522,1621.990149,NaN,NaN


## 4. Sentiment Analysis

If news data is available, let's analyze the sentiment of financial news.

In [13]:
# Perform sentiment analysis on news data
try:
    if 'news_data' in locals() and not news_data.empty:
        print("Analyzing news sentiment...")
        
        # Check if SentimentAnalyzer is available
        if 'SentimentAnalyzer' in globals():
            # Initialize sentiment analyzer
            sentiment_analyzer = SentimentAnalyzer(['vader', 'textblob'])
            
            # Perform sentiment analysis
            news_with_sentiment = sentiment_analyzer.analyze_dataframe(news_data)
            print(f"Added sentiment features. Shape: {news_with_sentiment.shape}")
            
            # Show sentiment distribution
            if 'vader_label' in news_with_sentiment.columns:
                print("\n VADER Sentiment Distribution:")
                vader_counts = news_with_sentiment['vader_label'].value_counts()
                print(vader_counts)
                
                # Create sentiment pie chart
                fig = px.pie(
                    values=vader_counts.values, 
                    names=vader_counts.index,
                    title="News Sentiment Distribution (VADER)",
                    color_discrete_sequence=px.colors.qualitative.Set3
                )
                fig.show()
            
            # Show sentiment over time
            if 'date' in news_with_sentiment.columns and 'vader_compound' in news_with_sentiment.columns:
                daily_sentiment = news_with_sentiment.groupby('date')['vader_compound'].mean().reset_index()
                
                fig = px.line(
                    daily_sentiment, 
                    x='date', 
                    y='vader_compound',
                    title='Average Daily News Sentiment Score',
                    labels={'vader_compound': 'Sentiment Score', 'date': 'Date'}
                )
                fig.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="Neutral")
                fig.show()
            
            # Sample sentiment analysis results
            print("\n📰 Sample News with Sentiment:")
            sample_cols = ['title', 'vader_label', 'vader_compound', 'textblob_label', 'textblob_polarity']
            available_cols = [col for col in sample_cols if col in news_with_sentiment.columns]
            display(news_with_sentiment[available_cols].head())
            
        else:
            print("SentimentAnalyzer not available")
            print(" Please run cell 2 first to import SentimentAnalyzer")
            print(" Creating basic sentiment analysis fallback...")
            
            # Basic sentiment analysis fallback using VADER only
            try:
                from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
                import pandas as pd
                
                analyzer = SentimentIntensityAnalyzer()
                news_with_sentiment = news_data.copy()
                
                # Apply VADER sentiment to each news item
                sentiments = []
                for text in news_data['title'].fillna(''):
                    if text:
                        scores = analyzer.polarity_scores(text)
                        sentiments.append({
                            'vader_compound': scores['compound'],
                            'vader_pos': scores['pos'],
                            'vader_neu': scores['neu'], 
                            'vader_neg': scores['neg']
                        })
                    else:
                        sentiments.append({
                            'vader_compound': 0.0,
                            'vader_pos': 0.0,
                            'vader_neu': 1.0,
                            'vader_neg': 0.0
                        })
                
                # Add sentiment scores to dataframe
                sentiment_df = pd.DataFrame(sentiments)
                news_with_sentiment = pd.concat([news_with_sentiment, sentiment_df], axis=1)
                
                # Add sentiment labels
                news_with_sentiment['vader_label'] = news_with_sentiment['vader_compound'].apply(
                    lambda x: 'positive' if x >= 0.05 else 'negative' if x <= -0.05 else 'neutral'
                )
                
                print(f" Basic sentiment analysis completed. Shape: {news_with_sentiment.shape}")
                print(f" Sentiment distribution: {news_with_sentiment['vader_label'].value_counts().to_dict()}")
                
                # Show sample results
                print("\n Sample News with Basic Sentiment:")
                display(news_with_sentiment[['title', 'vader_label', 'vader_compound']].head())
                
            except ImportError:
                print(" VADER sentiment library not available")
                print(" Install with: pip install vaderSentiment")
                import pandas as pd
                news_with_sentiment = pd.DataFrame()
        
    else:
        print(" No news data available for sentiment analysis")
        print(" To enable news sentiment analysis:")
        print(" 1. Make sure the news collection cell ran successfully")
        print(" 2. Check API keys in .env file")
        print(" 3. Re-run the news collection cell")
        
        # Create empty sentiment data for demonstration
        import pandas as pd
        news_with_sentiment = pd.DataFrame()
        
except NameError as e:
    print(f" Import error: {e}")
    print(" Please run cell 2 first to import all required modules")
    import pandas as pd
    news_with_sentiment = pd.DataFrame()
    
except Exception as e:
    print(f"Error in sentiment analysis: {e}")
    import pandas as pd
    news_with_sentiment = pd.DataFrame()

Analyzing news sentiment...
SentimentAnalyzer not available
 Please run cell 2 first to import SentimentAnalyzer
 Creating basic sentiment analysis fallback...
 Basic sentiment analysis completed. Shape: (442, 15)
 Sentiment distribution: {'neutral': 206, 'positive': 174, 'negative': 62}

 Sample News with Basic Sentiment:


,title,vader_label,vader_compound
0,Reliance Q2 results 2025: Mukesh Ambani-led RI...,neutral,0.0000
1,Reliance Industries Q2 result: Oil-to-telecom ...,neutral,0.0000
2,Explained: Reliance Industries is India’s most...,positive,0.2952
3,RIL shares fall over 2% post Q1 results; analy...,negative,-0.4023
4,Reliance Industries shares tumble: Why the con...,negative,-0.0531


## 5. Model Training and Evaluation

Now let's train machine learning models to predict stock movements.

In [14]:
# Prepare data for machine learning
if not stock_with_target.empty and len(stock_with_target) > 50:
    print("Preparing data for machine learning...")
    
    # Integrate stock and news data if both available
    if not news_with_sentiment.empty:
        print("Integrating stock and news data...")
        integrator = DataIntegrator()
        try:
            ml_data = integrator.align_data_by_date(stock_with_target, news_with_sentiment)
            print(f" Integrated data shape: {ml_data.shape}")
        except KeyError as e:
            print(f"Error during integration: {e}")
            print("Ensure all required columns are present in news_with_sentiment.")
            ml_data = stock_with_target.copy()
    else:
        print(" Using stock data only...")
        ml_data = stock_with_target.copy()
    
    # Prepare features for ML
    stock_preprocessor = StockDataPreprocessor()
    try:
        ml_ready_data = stock_preprocessor.prepare_features(ml_data)
        
        # Remove rows with missing targets
        ml_ready_data = ml_ready_data.dropna(subset=['target'])
        
        print(f" ML-ready data shape: {ml_ready_data.shape}")
        print(f" Final target distribution:")
        print(ml_ready_data['target'].value_counts())
    except Exception as e:
        print(f"Error during feature preparation: {e}")
        ml_ready_data = pd.DataFrame()
    
    # Check if all classes have at least 2 samples
    if not ml_ready_data.empty:
        target_counts = ml_ready_data['target'].value_counts()
        if (target_counts < 2).any():
            print(" Not enough samples in one or more classes. Each class must have at least 2 samples.")
            print("Consider collecting more data or adjusting the target variable.")
        elif len(ml_ready_data) >= 50:
            # Train a simple logistic regression model for demonstration
            print("\n Training Logistic Regression model...")
            
            lr_model = LogisticRegressionModel()
            try:
                metrics = lr_model.train(ml_ready_data, target_col='target')
                
                print("\n Model Performance:")
                for metric, value in metrics.items():
                    print(f"  {metric.capitalize()}: {value:.4f}")
                
                # Get feature importance
                if lr_model.is_fitted:
                    feature_importance = lr_model.get_feature_importance()
                    
                    print("\n🔝 Top 10 Most Important Features:")
                    display(feature_importance.head(10))
                    
                    # Plot feature importance
                    top_features = feature_importance.head(15)
                    
                    fig = px.bar(
                        top_features,
                        x='importance',
                        y='feature',
                        orientation='h',
                        title='Top 15 Most Important Features',
                        labels={'importance': 'Feature Importance', 'feature': 'Feature'}
                    )
                    fig.update_layout(height=500)
                    fig.show()
            except Exception as e:
                print(f"Error during model training: {e}")
        else:
            print(f"Not enough data for training. Need at least 50 samples, got {len(ml_ready_data)}")
    else:
        print(" No processed stock data available for model training")
else:
    print(" No processed stock data available for model training")
    print(" Make sure stock data collection was successful")

INFO:src.preprocessing:Aligning stock and news data by date...
INFO:src.preprocessing:Integrated data shape: (634, 79)
INFO:src.preprocessing:Preparing features for ML...
INFO:src.preprocessing:Integrated data shape: (634, 79)
INFO:src.preprocessing:Preparing features for ML...
INFO:src.preprocessing:Prepared 69 features for ML
INFO:src.preprocessing:Prepared 69 features for ML


Preparing data for machine learning...
Integrating stock and news data...
 Integrated data shape: (634, 79)
 ML-ready data shape: (634, 79)
 Final target distribution:
target
DOWN       313
UP         307
NEUTRAL     14
Name: count, dtype: int64

 Training Logistic Regression model...


NameError: name 'LogisticRegressionModel' is not defined

## 6. Technical Analysis Visualizations

Let's create some technical analysis charts to understand stock patterns better.

In [ ]:
# Create technical analysis charts
if 'stock_with_indicators' in locals() and not stock_with_indicators.empty:
    # Select one stock for detailed technical analysis
    selected_symbol = stock_with_indicators['symbol'].iloc[0]
    symbol_data = stock_with_indicators[stock_with_indicators['symbol'] == selected_symbol].copy()
    symbol_data = symbol_data.sort_values('date')
    
    print(f" Creating technical analysis charts for {selected_symbol}")
    
    # Create comprehensive technical analysis chart
    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(
            f'{selected_symbol.replace(".NS", "")} - Price with Moving Averages',
            'RSI (Relative Strength Index)',
            'MACD',
            'Volume'
        ),
        row_heights=[0.4, 0.2, 0.2, 0.2]
    )
    
    # Row 1: Price with SMA and EMA
    fig.add_trace(go.Scatter(
        x=symbol_data['date'], y=symbol_data['close'],
        mode='lines', name='Close Price', line=dict(color='blue', width=2)
    ), row=1, col=1)
    
    if 'sma_20' in symbol_data.columns:
        fig.add_trace(go.Scatter(
            x=symbol_data['date'], y=symbol_data['sma_20'],
            mode='lines', name='SMA 20', line=dict(color='orange', width=1)
        ), row=1, col=1)
    
    if 'ema_12' in symbol_data.columns:
        fig.add_trace(go.Scatter(
            x=symbol_data['date'], y=symbol_data['ema_12'],
            mode='lines', name='EMA 12', line=dict(color='green', width=1)
        ), row=1, col=1)
    
    # Row 2: RSI
    if 'rsi' in symbol_data.columns:
        fig.add_trace(go.Scatter(
            x=symbol_data['date'], y=symbol_data['rsi'],
            mode='lines', name='RSI', line=dict(color='purple')
        ), row=2, col=1)
        
        # Add RSI levels
        fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)
    
    # Row 3: MACD
    if 'macd' in symbol_data.columns:
        fig.add_trace(go.Scatter(
            x=symbol_data['date'], y=symbol_data['macd'],
            mode='lines', name='MACD', line=dict(color='blue')
        ), row=3, col=1)
        
        if 'macd_signal' in symbol_data.columns:
            fig.add_trace(go.Scatter(
                x=symbol_data['date'], y=symbol_data['macd_signal'],
                mode='lines', name='MACD Signal', line=dict(color='red')
            ), row=3, col=1)
    
    # Row 4: Volume
    fig.add_trace(go.Bar(
        x=symbol_data['date'], y=symbol_data['volume'],
        name='Volume', marker_color='lightblue'
    ), row=4, col=1)
    
    # Update layout
    fig.update_layout(
        title=f'Technical Analysis - {selected_symbol.replace(".NS", "")}',
        height=800,
        showlegend=True,
        hovermode='x unified'
    )
    
    # Update y-axis labels
    fig.update_yaxes(title_text="Price (₹)", row=1, col=1)
    fig.update_yaxes(title_text="RSI", row=2, col=1, range=[0, 100])
    fig.update_yaxes(title_text="MACD", row=3, col=1)
    fig.update_yaxes(title_text="Volume", row=4, col=1)
    
    fig.show()
    
    # Daily Returns Distribution
    if 'daily_return' in symbol_data.columns:
        fig_returns = px.histogram(
            symbol_data,
            x='daily_return',
            nbins=50,
            title=f'Daily Returns Distribution - {selected_symbol.replace(".NS", "")}',
            labels={'daily_return': 'Daily Return', 'count': 'Frequency'}
        )
        fig_returns.add_vline(x=0, line_dash="dash", line_color="red", annotation_text="Zero Return")
        fig_returns.show()
        
        # Calculate some statistics
        returns = symbol_data['daily_return'].dropna()
        print(f"\\n Daily Returns Statistics for {selected_symbol}:")
        print(f"  Mean: {returns.mean():.4f}")
        print(f"  Std Dev: {returns.std():.4f}")
        print(f"  Skewness: {returns.skew():.4f}")
        print(f"  Min: {returns.min():.4f}")
        print(f"  Max: {returns.max():.4f}")
else:
    print(" No technical indicators data available for visualization")

 Creating technical analysis charts for RELIANCE.NS


\n Daily Returns Statistics for RELIANCE.NS:
  Mean: -0.0384
  Std Dev: 0.3697
  Skewness: -0.0648
  Min: -0.6197
  Max: 0.5475


## 7. Next Steps and Conclusions

This notebook demonstrated the complete workflow of the NEWS2PROFIT project:

### What We've Accomplished:
1. **Data Collection**: Fetched stock data from yfinance and news data from multiple sources
2.  **Data Preprocessing**: Cleaned data and created technical indicators (SMA, EMA, RSI, MACD)
3.  **Feature Engineering**: Generated predictive features and target variables
4. **Sentiment Analysis**: Analyzed news sentiment using VADER and TextBlob
5.  **Model Training**: Trained machine learning models for stock movement prediction
6. **Visualization**: Created interactive charts for analysis

### Key Findings:
- Technical indicators provide valuable signals for stock movement prediction
- News sentiment can add additional predictive power when available
- Multiple models can be compared to find the best performing approach

### Next Steps:
1. **Add More Data Sources**: Include more financial news APIs and social media data
2. **Advanced Models**: Implement XGBoost and LSTM models for better performance  
3. **Real-time Predictions**: Set up automated daily predictions
4. **Risk Management**: Add portfolio optimization and risk metrics
5. **Dashboard Deployment**: Launch the Streamlit dashboard for live monitoring

### How to Run the Complete Pipeline:
```bash
# 1. Set up environment variables
cp .env.template .env
# Edit .env with your API keys

# 2. Run the full training pipeline
python train_models.py

# 3. Launch the interactive dashboard
streamlit run src/dashboard.py

# 4. Run tests
python -m pytest tests/
```

### Resources:
-  [Project Documentation](../README.md)
-  [Configuration Settings](../config/config.py)
-  [Unit Tests](../tests/test_modules.py)
-  [Dashboard](../src/dashboard.py)